## 机器学习基石 Assignment1 Q16-18

Q16. First, we use an artificial data set to study `PLA`. The data set is in

https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw1_15_train.dat

Each line of the data set contains one ($x_n,y_n$) with $x_n\in ℝ^4$. The first 4 numbers of the line contains the components of $x_n$ orderly, the last number is $y_n$.

Please initialize your algorithm with $w=0$ and take sign(0) as −1.

Implement a version of PLA by visiting examples in the naive cycle using the order of examples in the data set. Run the algorithm on the data set. What is the number of updates before the algorithm halts?



In [44]:
import numpy as np
#line profiler
%load_ext line_profiler 
import matplotlib.pyplot as plt

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [45]:
data = np.loadtxt('hw1_15_train.dat',dtype=np.float32)
(n, nx) = data.shape
x = np.hstack((np.ones((n,1)), data[:,0:4])).T
y = data[:,4].astype(int)

In [46]:
w = np.zeros((5,))

def sign(w, x):
    """
    Sign of w.T*x
    """
    if np.dot(w.T, x)>0:
        return 1
    else:
        return -1

def check_error(w, x, i):
    """
    check error sequentially from position i to find error
    return index of x that causes error
    """
    _,n = x.shape
    for j in range(i+1, n):
        if sign(w, x[:,j]) != y[j]:
            return (True, j)
    for j in range(i):
        if sign(w, x[:,j]) != y[j]:
            return (True, j)
    return (False, 0)

def pla(w, x, y):
    """
    Perceptron Learning Algorithm(PLA)
    Input: 
    w, size: nx, 1
    x, size: nx, n
    y, size: n, 1
    """
    update_count = 0
    error = check_error(w,x,0)
    while error[0]: #while no error in PLA, stops
        i = error[1] # get index
        w = w + (y[i]*x[:,i])
        error = check_error(w, x, i)
        update_count += 1
    return w, update_count


w, update_count = pla(w, x, y)
update_count

37

Q17. Implement a version of PLA by visiting examples in fixed, pre-determined random cycles throughout the algorithm. Run the algorithm on the data set. Please repeat your experiment for 2000 times, each with a different random seed. What is the average number of updates before the algorithm halts?

In [47]:
def find_a_mistake(w, x, i, visiting):
    """
    check error sequentially from position i to find error
    return index of x that causes error
    """
    _, n = x.shape
        
    # visiting
    for j in visiting[i:]:
        if sign(w, x[:,j]) != y[j]:
            return (True, j)
    for j in visiting[:i]:
        if sign(w, x[:,j]) != y[j]:
            return (True, j)
    return (False, 0)

def random_pla(w, x, y, visiting):
    """
    Perceptron Learning Algorithm(PLA)
    Randomly visit input 
    Input: 
    w, size: nx, 1
    x, size: nx, n
    y, size: n, 1
    """
    update_count = 0
    error = find_a_mistake(w, x, 0, visiting)
    while error[0]: #while no error in PLA, stops
        i = error[1] # get index
        w = w + (y[i]*x[:,i])
        error = find_a_mistake(w, x, i, visiting)
        update_count += 1
    return w, update_count


#visit 2000 times
seed = 1; # set seed
update_count_sum = 0
for i in range(2000):
    ##visiting
    np.random.seed(seed)
    visiting = np.arange(n)
    np.random.shuffle(visiting)
    
    w = np.zeros((5,))
    w, update_count = random_pla(w, x, y, visiting)
    update_count_sum += update_count
    seed += 1 # change seed to produce difference visiting
    
update_count = update_count_sum/2000
print(update_count)

40.829


Q18. Next, we play with the pocket algorithm. Modify your PLA in Question 16 to visit examples purely randomly, and then add the "pocket" steps to the algorithm. We will use

https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw1_18_train.dat

as the training data set $\mathcal{D}$, and

https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw1_18_test.dat

as the test set for "verifying'' the g returned by your algorithm (see lecture 4 about verifying). The sets are of the same format as the previous one. Run the pocket algorithm with a total of 50 updates on $\mathcal{D}$ , and verify the performance of $w_{POCKET}$ using the test set. Please repeat your experiment for 2000 times, each with a different random seed. What is the average error rate on the test set?

In [48]:
## data preparation
data = np.loadtxt('hw1_18_train.dat',dtype=np.float32)
(n, nxy) = data.shape
x = np.hstack((np.ones((n,1)), data[:,0:nxy-1])).T
y = data[:,nxy-1].astype(int)

## test data
data_test = np.loadtxt('hw1_18_test.dat',dtype=np.float32)
(ntest, nxytest) = data.shape
xtest = np.hstack((np.ones((ntest,1)), data[:,0:nxytest-1])).T
ytest = data[:,nxytest-1].astype(int)

n, ntest

(500, 500)

In [69]:
def predict(w, x, y):
    """
    Predicting error using vectorization
    """
    error_vector = ((np.dot(w.T, x)>0) != (y==1))
    return sum(error_vector)/x.shape[1]


def pocket(w, x, y, visiting, learning_rate, update_count_limit, count_limit):
    """
    Pocket Algorithm
    Randomly visiting(set by seed)
    Input: 
    w, size: nx, 1
    x, size: nx, n
    y, size: n, 1
    """
    error_rate = 1
    update_count = 0
    error = find_a_mistake(w, x, 0, visiting)
    count = 0
    once_count = 0
    #while no error in Pocket or after 50 updates, stops
    # if count > n, all possible w are used.
    while (error[0] and (update_count < update_count_limit) and (count < count_limit)): 
        i = error[1] # get index
        w = w + learning_rate*(y[i]*x[:,i])
        error_temp = predict(w, x, y)
        if  error_temp < error_rate:
            best_w = w
            error_rate = error_temp 
            update_count += 1
            once_count = 0
        error = find_a_mistake(w, x, i, visiting)
        count += 1
        once_count += 1
        if once_count > x.shape[1]:
            return best_w
    return best_w

def model(x, y, xtest, ytest, N=2000, learning_rate=0.01,
          update_count_limit=50, count_limit=float("inf")):
    """
    Pocket Model
    """
    seed = 1; # set seed
    error_rate = []; #error_rate
    #visit N times
    for i in range(N):
        ##visiting
        np.random.seed(seed)
        visiting = np.arange(n)
        np.random.shuffle(visiting)

        w = np.zeros((5,))
        w = pocket(w, x, y, visiting, learning_rate, update_count_limit, count_limit)
        error_rate.append(predict(w, xtest, ytest))
        seed += 1 # change seed to produce difference visiting
        if (i%10 == 0):
            print('STEP= ', i)
    print("average error=", np.average(error_rate))
    
model(x, y, xtest, ytest, N=50) #N=100 costs less than 1 minutes

STEP=  0
STEP=  10
STEP=  20
STEP=  30
STEP=  40
average error= 0.09884



Modify your algorithm in Q18 to return $w_{50}$ instead of $\hat{w}$ (the pocket vector) after 50 updates.

Run the modified algorithm on $\mathcal{D}$, and verify the performance using the test set.

Please repeat your experiment for 2000 times, each with a different random seed. What is the average error rate on the test set?

In [70]:
#set update_count_limit > count_limit
model(x, y, xtest, ytest, N=50, learning_rate=0.01, 
      update_count_limit=51, count_limit=50)

STEP=  0
STEP=  10
STEP=  20
STEP=  30
STEP=  40
average error= 0.124
